In [2]:
#|default_exp export

## Exporting Tests to Files

In [223]:
#|export
from nbprocess.process import *
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.maker import *

In [21]:
everything_fn = "00_core.ipynb"

In [226]:
#|export
from collections import defaultdict
from fastcore.foundation import L, ifnone
from execnb.nbio import *

class ExportTestProc:
    def __init__(self): self.tests = defaultdict(L)
    def _test_(self, nbp, cell, exp_to=None):
        self.tests[f'test_{ifnone(exp_to, "#")}'].append(nbp.cell)

In [219]:
#|export
def write_test_cells(cells, hdr, file, offset=0, name=""):
    "Writes the starter import and writes an initial unittest"
    file.write(f'\nfrom {get_config().lib_name}.{name.lower()} import *')
    file.write(f'\nimport unittest\n')
    file.write(f'\nclass {name.title()}Tester(unittest.TestCase):')
    for cell in cells:
        if cell.source.strip():
            if "import" in cell["directives_"]["test"]:
                # Expected, this should be at the top. What happens is its never written
                content = f'\n{hdr} {cell.idx_+offset}\n{cell.source}\n'
            else:
                test_name = cell["directives_"]["test"][0]
                content = '\n'.join([f"\t\t{c}" for c in cell.source.split("\n")])
                content = f'\n{hdr} {cell.idx_+offset}\n\tdef test_{test_name}(self):\n{content}\n'
            file.write(content)

In [225]:
#|export
import ast
class TestMaker(ModuleMaker):
    "Module maker that will write test cells depending on a flag"
    def make_all(self, cells): pass
    def _make_exists(self, cells, all_cells=None, lib_name:str="", test_cells:bool=False):
        "`make` for `is_new=False`"
        with self.fname.open("a") as f: write_test_cells(cells, self.hdr, f)
    
    def make(self, cells, all_cells=None, lib_name=None, test_cells:bool=False):
        if lib_name is None: lib_name = get_config().lib_name
        if all_cells is None: all_cells = cells
        for cell in all_cells: cell.import2relative(lib_name)
        if not self.is_new: return self._make_exists(cells, all_cells, lib_name, test_cells)
        
        self.fname.parent.mkdir(exist_ok=True, parents=True)
        trees = cells.map(NbCell.parsed_)
        try: last_future = max(i for i,tree in enumerate(trees) if tree and any(
             isinstance(t,ast.ImportFrom) and t.module=='__future__' for t in tree))+1
        except ValueError: last_future=0
        with self.fname.open('w') as f:
            f.write(f"# AUTOGENERATED! DO NOT EDIT! File to edit: {self.dest2nb}.")
            write_cells(cells[:last_future], self.hdr, f, 0)
            write_test_cells(cells, self.hdr, f, 0, self.name.replace("test_","").capitalize())
            f.write('\n')

In [222]:
exp = ExportTestProc()
nb = NBProcessor(everything_fn, exp)
nb.process()
for _, cells in exp.tests.items():
    name = "test_core"
    mm = TestMaker(dest='tests', name=name, nb_path=everything_fn, is_new=True)
    mm.make(cells, test_cells=False)